In [35]:
import pandas as pd 
import json
import re
import numpy as np 
import torch
import tqdm


## Review.json을 text file로

In [3]:
data = []
with open('./Librarything/reviews.json', 'r', encoding='UTF-8') as f:
   for line in f:
        line = json.dumps(line)
        data.append(json.loads(line))

In [4]:
item = pd.DataFrame(data)

In [5]:
temp = []
exception = []
for index,value in item.iterrows():
    first = value[0].find("'comment'")
    second = value[0].find("'user'")
    content = value[0][first+12:second-3].replace("\"","'")
    content = content.replace("\\",'')
    review = "\""+content+"\""
    change = value[0].replace("'","\"")
    word = change[:first+11]+review+change[second-2:]
    try:
        temp.append(json.loads(word))
    except:
        exception.append([index, word])

In [6]:
dataframe = pd.DataFrame(temp)

In [7]:
dataframe.head()

,work,flags,unixtime,stars,nhelpful,time,comment,user
0,3206242,[],1194393600,5.0,0,"Nov 7, 2007",This a great book for young readers to be intr...,van_stef
1,12198649,[],1333756800,5.0,0,"Apr 7, 2012",Help Wanted: Tales of On The Job Terror from E...,dwatson2
2,12533765,[],1352937600,NaN,0,"Nov 15, 2012","Magoon, K. (2012). Fire in the streets. New Yo...",edspicer
3,12981302,[],1364515200,4.0,0,"Mar 29, 2013","Well, I definitely liked this book better than...",amdrane2
4,5231009,[],1270944000,3.0,0,"Apr 11, 2010",It's a nice science-fiction thriller with some...,Lila_Gustavus


In [29]:
trust = pd.read_table('./Librarything/edges.txt', sep=' ', header =None, names=['friend1','friend2'])

In [30]:
trust.head()

,friend1,friend2
0,Rodo,anehan
1,Rodo,sevilemar
2,Rodo,dingsi
3,Rodo,slash
4,RelaxedReader,AnnRig


In [32]:
print("총 Edge의 수 : ", len(trust))
unique_trust = set()
unique_trust.update(trust['friend1'].unique())
unique_trust.update(trust['friend2'].unique())
print("Edge에 참여하는 User의 수 : ", len(unique_trust))

총 Edge의 수 :  219790
Edge에 참여하는 User의 수 :  65946


In [27]:
print("전처리 이전 User의 수 : ", len(dataframe['user'].unique()))
print("전처리 이전 Item의 Unique 수 : ",len(dataframe['work'].unique()))
print("전처리 이전 interaction의 수 : ", len(dataframe))

전처리 이전 User의 수 :  83195
전처리 이전 Item의 Unique 수 :  506165
전처리 이전 interaction의 수 :  1706979


## 전처리

In [28]:
import pickle
with open('./Librarything/library_review.pickle', 'wb') as f:
    pickle.dump(dataframe,f)

## user에게 index만들고 저장

In [33]:
df = dataframe

In [34]:
unique_user = set(df['user'])
map_user = []
delete_user = []
index = 0
for user in tqdm.tqdm(unique_user,'set'):
    match_user = df[df['user']==user]
    if len(match_user) > 1:
        map_user.append([user, index])
        index+=1
    else:
        delete_user.append(user)
        df.drop(match_user.index, inplace=True)
map_user_df = pd.DataFrame(map_user, columns = ['org_id', 'remap_id'])

NameError: name 'tqdm' is not defined

In [ ]:
print("review를 남긴 user의 수 : ", len(map_user))

In [ ]:
map_user_df.to_csv('./Librarything/user_review_list.txt', sep = '\t', index =False)

In [ ]:
trust_user_index = []
map_user_set = set(map_user_df['org_id'])
for t in tqdm.tqdm(unique_trust, 'set'):
    if t not in map_user_set:
        trust_user_index.append([t, index])
        index+=1
trust_user_df = pd.DataFrame(trust_user_index, columns = ['org_id', 'remap_id'])
map_user_df_2 = pd.concat([map_user_df, trust_user_df], ignore_index=True)

In [ ]:
print("Edge와 Review를 남긴 총 Unique User의 수 : ", len(map_user_df_2))

In [ ]:
map_user_df_2.to_csv('./Librarything/user_list.txt', sep = '\t', index =False)

## item에 index를 부여

In [ ]:
map_list = []
item_list = df['work'].unique()
delete_item = []
index = 0
for item in tqdm.tqdm(item_list, desc='list'):
        map_list.append([item, index])
        index+=1
map_df = pd.DataFrame(map_list, columns=['org_id', 'remap_id'])
map_df.to_csv('./Librarything/item_list.txt', sep = '\t', index = False)

In [ ]:
map_user_dict = dict(zip(map_user_df.org_id,map_user_df.remap_id))
map_item_dict = dict(zip(map_df.org_id,map_df.remap_id))
map_id_item_dict = dict(zip(map_df.remap_id, map_df.org_id))

In [ ]:
u_i_rating = df[['user', 'item']]
u_i_rating.head()

In [ ]:
u_i_rating['user'] = u_i_rating['user'].map(lambda x: map_user_dict[x])
u_i_rating['item'] = u_i_rating['item'].map(lambda x: map_item_dict[x])
u_i_rating = np.asarray(u_i_rating)

In [ ]:
interaction = []
map_user_ar = sorted(map_user_dict.values())
for user_id in map_user_ar:
    match = u_i_rating[u_i_rating[:,0] == user_id]
    interaction.append([user_id]+list(match[:,1]))

In [ ]:
item_set = set()
for l in interaction:
    item_set.update(l[1:])

In [ ]:
print("Edge에 참여하고 Review를 남긴 Unique user의 수 : ", len(map_user_df_2))
print("Edge에만 참여하는 Unique user의 수 : ", len(trust))
print("Review만 남긴 Unique user의 수 : ",  83195)
print()
print("review를 2개이상 남긴 user의 Unique 수", len(map_user))
print("Unique item의 개수 : ",len(item_set))
print()
print("전처리하기 전의 Unique item의 수 : ", 506165)
print()



## total_data_2.txt는 index 조절한 것

In [ ]:
with open('./Librarything/total_data_2.txt', 'w') as output:
    for line in interaction:
        output.write(" ".join(map(str,line)))
        output.write('\n')

##  train(0.8)과 test(0.2) 나누기

In [ ]:
train_set = {}
test_set = {}
with open('./Librarything/total_data_2.txt', 'r') as output:
    for l in output.readlines():
        l = l.strip('\n').split(' ')
        items = [int(i) for i in l[1:]]
        uid = int(l[0])
        train_length = floor(len(items)*0.8)
        train_set[uid] = random.sample(items, train_length)
        test_set[uid] = list(set(items) - set(train_set[uid]))
    

In [ ]:
with open('./Librarything/train.txt', 'w') as output:
    for k, v in train_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')
with open('./Librarything/test.txt', 'w') as output:
    for k, v in test_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')